<a href="https://colab.research.google.com/github/berrygayo/BigdataAnalysisCertification/blob/main/%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D(%EB%A6%BF%EC%A7%80%2C%EB%9D%BC%EC%8F%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 선형 회귀 (최소 제곱법)
+ 평균제곱오차를 최소화하는 파라미터 w,b를 찾는다. 

In [ ]:
#!pip install mglearn
import mglearn

In [ ]:
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
X, y  = mglearn.datasets.make_wave(n_samples=60)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

lr = LinearRegression().fit(X_train, y_train)

In [ ]:
print("lr.coef_:", lr.coef_) # w
print("lr.intercept_:", lr.intercept_) # b

In [ ]:
print("훈련 세트 점수: {:.2f}".format(lr.score(X_train, y_train)))
print("테스트 세트 점수: {:.2f}".format(lr.score(X_test, y_test)))

# R^2 값이 그닥 높지 않다.
# 훈련세트와 테스트세트의 점수가 비슷 : 과대적합이 아니라 과소 적합인 상태를 뜻함 

### 그렇다면 선형회귀모델이 보스턴 주택 가격과 같은 복잡한 데이테에서는 어떻게 적용하는가 ? 
( 샘플 506개, 특성은 유도된 것을 합쳐 104개 )

In [ ]:
X, y = mglearn.datasets.load_extended_boston()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lr = LinearRegression().fit(X_train, y_train)

In [ ]:
print("훈련 세트 점수: {:.2f}".format(lr.score(X_train, y_train)))
print("테스트 세트 점수: {:.2f}".format(lr.score(X_test, y_test)))

# 훈련 세트에 과적합됨 > 억제 필요 > 릿지나 라쏘 

## 릿지 회귀 
+ L2 규제 (L2 norm)
+ 가중치 파라미터(w)를 0에 가깝게
+ 유클리드 사용 

In [ ]:
from sklearn.linear_model import Ridge 
ridge = Ridge().fit(X_train, y_train) # alpha 지정 안할 시 기본 1로 설정됨 
print("훈련 세트 점수 : {:.2f}".format(ridge.score(X_train, y_train)))
print("테스트 세트 점수: {:.2f}".format(ridge.score(X_test,y_test)))

# 기존 선형 모델의 테스트 점수 : 0.69 보다 높아짐 
# 선형회귀는 테스트 셋에 과적합 되지만 릿지는 덜 그럼 

In [ ]:
ridge10 = Ridge(alpha=10).fit(X_train, y_train) # alpha를 높이면 계수를 더 0에 가깝게 만듬 > 훈련세트의 성능 낮아지지만, 일반화에는 도움 
print("훈련 세트 점수: {:.2f}".format(ridge10.score(X_train, y_train)))
print("테스트 세트 점수: {:.2f}".format(ridge10.score(X_test, y_test)))

In [ ]:
ridge01 = Ridge(alpha=0.1).fit(X_train, y_train)
print("훈련 세트 점수: {:.2f}".format(ridge01.score(X_train, y_train)))
print("테스트 세트 점수: {:.2f}".format(ridge01.score(X_test, y_test)))

In [ ]:
import matplotlib.pyplot as plt

# alpha 값 변경에 따른 coef_속성 변경 확인하기 
plt.plot(ridge10.coef_, '^', label="Ridge alpha=10")
plt.plot(ridge.coef_, 's', label="Ridge alpha=1")
plt.plot(ridge01.coef_, 'v', label="Ridge alpha=0.1")

plt.plot(lr.coef_, 'o', label="LinearRegression")
plt.xlabel("계수 목록")
plt.ylabel("계수 크기")
xlims = plt.xlim()
plt.hlines(0, xlims[0], xlims[1])
plt.xlim(xlims)
plt.ylim(-25, 25)
plt.legend()
plt.show() # 책에는 없음

> 그래프 해석 
+ x 축은 coef_의 원소를 위치대로 나열한 것 
+ x=0은 첫 번째 특성에 연관된 계수 
+ x=1은 두 번째 특성에 연관된 계수, 이런 식으로 x=100까지 계속 된다. 
+ y 축은 각 계수의 수치, alpha=10 일 때 대부분의 계수는 -3과 3사이에 위치 
+ alpha = 0.1 일 때 계수는 더 커지며 
+ alpha=0 규제가 없는 선형 회귀의 계수는 값이 더 커져 그림 밖으로 넘어간다. 

## 학습 곡선 그리기 
+ 데이터셋의 크기에 따른 모델의 성능 변화 

In [ ]:
mglearn.plots.plot_ridge_n_samples()
# 선형회귀랑 , alpha =1 일때 Ridge 적용 

> 그래프 해석 
+ 훈련 세트에 대해선 두 모델 다 처음부터 점수가 높다.
+ 훈련 세트에서 선형 회귀 덤수가 리지보다 높은데, 이는 규제가 없기 때문 
+ 테스트 세트에선 릿지가 선형회귀보다 점수가 높으며, 선형의 경우 n=400 미만에선 아무것도 학습 못하고 있음 
+ 두 모델 모두 데이터가 많이질수록 좋아지고 마지막에는 선형회귀가 릿지를 따라잡는다. 
+ 선형 회귀의 훈련 데이터는 성능이 감소 : 데이터가 많아질수록 모델이 데이터를 기억하거나 과대적합하기 어려워지기 때문 

## 라쏘 회귀 
+ L1 규제 
+ 가중치를 0으로 
+ 모델에서 완전히 제외되는 특성이 생길 수 있다 
+ 멘하탄 거리 사용 

In [ ]:
import numpy as np

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso().fit(X_train, y_train)
print("훈련 세트 점수: {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 세트 점수: {:.2f}".format(lasso.score(X_test, y_test)))
print("사용한 특성의 개수:", np.sum(lasso.coef_ != 0))

# 104개 특성중 4개만 사용 
# 훈련세트와 테스트 세트 모두 점수가 낮음 

In [ ]:
# 과소적합을 줄이기 위해 alpha 값을 줄여보자 
# 이렇게 하려면 max_iter(반복 실행하는 최대 횟수)의 기본값을 늘려야 한다. 

# max_iter 기본 값을 증가시키지 않으면 max_iter 값을 늘이라는 경고가 발생합니다
lasso001 = Lasso(alpha=0.01, max_iter=50000).fit(X_train, y_train)
print("훈련 세트 점수: {:.2f}".format(lasso001.score(X_train, y_train)))
print("테스트 세트 점수: {:.2f}".format(lasso001.score(X_test, y_test)))
print("사용한 특성의 개수:", np.sum(lasso001.coef_ != 0))

# 알파값을 낮추면 모델의 복잡도는 증가하여 성능 좋아짐 
# 사용한 특성은 104개 중 33개 

In [ ]:
lasso00001 = Lasso(alpha=0.0001, max_iter=50000).fit(X_train, y_train)
print("훈련 세트 점수: {:.2f}".format(lasso00001.score(X_train, y_train)))
print("테스트 세트 점수: {:.2f}".format(lasso00001.score(X_test, y_test)))
print("사용한 특성의 개수:", np.sum(lasso00001.coef_ != 0))
# 알파값을 너무 낮추면 과대적합 발생 

In [ ]:
# 알파값의 따른 모델의 계수 그래프 
plt.plot(lasso.coef_, 's', label="Lasso alpha=1")
plt.plot(lasso001.coef_, '^', label="Lasso alpha=0.01")
plt.plot(lasso00001.coef_, 'v', label="Lasso alpha=0.0001")

plt.plot(ridge01.coef_, 'o', label="Ridge alpha=0.1")
plt.legend(ncol=2, loc=(0, 1.05))
plt.ylim(-25, 25)
plt.xlabel("계수 목록")
plt.ylabel("계수 크기")
plt.show() # 책에는 없음

> 그래프 해석 
+ alpha =1 일때 계수 대부분이 0이고, 나머지 계수들도 크기가 작다 
+ alpha =0.01 로 줄이면 대부분의 특성이 0이 되는 분포를 얻게 된다.
+ alpha=0.0001이 되면 계수 대부분이 0이 아니고 값도 커져 꽤 규제받지 않는 모델이 된다

+ alpha=0.1 인 릿지 모델은 alpha=0.01인 라소 모델과 성능이 비슷하지만 릿지를 사용하면 어느 계수도 0이 되지 않는다,.


실제로 이 두 모델 중 보통은 리지 회귀를 선호한다, 하지만 특성이 많고 그중 일부분만 중요하다면
라쏘가 더 좋은 선택일 수 있다.
또한 분석하기 쉬운 모델을 원한다면 라쏘가 입력 특성 중 일부만 사용하므로 쉽게 해석할 수 있는 모델을 만들어 줄 것 
사이키런은 라쏘와 릿지의 패널티를 결합한 엘라스틱넷도 제공 



# 분류에 대한 선형 모델 
+ 함수값 < 0 : -1, 함수값 > 0 : 1 로 예측 

> 두 가지 방법으로 구분 
1. 특정 계수와 절편의 조합이 훈련 데이터에 얼마나 잘 맞는지 
2. 사용할 수 있는 규제가 있는지, 있다면 어떤 방식인지 

## 로지스틱 회귀(LogisticRegression) & 서보트 벡터 머신 (SVM)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

X, y = mglearn.datasets.make_forge()

fig, axes = plt.subplots(1, 2, figsize=(10, 3))

for model, ax in zip([LinearSVC(max_iter=5000), LogisticRegression()], axes):
    clf = model.fit(X, y)
    mglearn.plots.plot_2d_separator(clf, X, fill=False, eps=0.5,
                                    ax=ax, alpha=.7)
    mglearn.discrete_scatter(X[:, 0], X[:, 1], y, ax=ax)
    ax.set_title(clf.__class__.__name__)
    ax.set_xlabel("특성 0")
    ax.set_ylabel("특성 1")
axes[0].legend()
plt.show() # 책에는 없음

> 그림 해석 
+ 첫 번째 특성을 x축, 두 번쨰 특성을 y축에 둠 
+ 직선 위는 1, 아래는 0 클래스로 분류 
+ 두 모델 다 비슷하게 분류하였고, 똑같이 두 포인트를 잘못 분류 
+ 이 두 모델 모두 L2 규제 사용 
+ 두 모델 모두 규제의 강도를 결정하는 매개변수는 C 이다. 
+ C 값이 높아지면 규제 감소 
+ C 값이 낮아지면 데이터 포인트 주 ㅇ다수에 맞추려고 하고, C 값을 높이면 개개의 데이터 포인트를 정확히 분류할 것 

In [ ]:
mglearn.plots.plot_linear_svc_regularization()

> 그래프 해석 
+ 왼쪽 그림은 아주 작은 C 값 때문에 규제가 많이 적용 
+ 중간 그림은 C값이 조금 더 크며 잘못 분류한 두 샘플에 민감해져 결정 경계가 기울어졌습니다. 
+ 오른쪽 그림에서 C값을 아주 크게 하였더니 결정 경계는 더 기울었고 마침내 클래스 0의 모든 데이터 포인트를 올바로 분류 했다.
+ 오른쪽 그림의 모델은 모든 데이터 포인트를 정확하게 분류하려고 애썼지만 클래스의 전체적인 배치를 잘 파악하지 못했다.( 과대적합 ) 


분류에서의 선형 모델은 낮은 차원의 데이터에서는 결정 경계가 직선이거나 평면이어서 매우 제한적인 것 처럼 보인다.
하지만 고차원에서는 분류에 대한 선형 모델이 매우 강력해지며 특성이 많아지면 과대적합되지 않도록 하는 것이 매우 중요해진다.
특성이 맣아지면 과대적합되지 않도록 하는 것이 매우 중요해진다.

## 유방암 데이터 셋을 사용한 LogisticRegression 

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify=cancer.target, random_state=42)
logreg = LogisticRegression(max_iter=5000).fit(X_train, y_train)
print("훈련 세트 점수: {:.3f}".format(logreg.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg.score(X_test, y_test)))

기본값 C=1 이 훈련 세트와 테스트 세트 양쪽에 95% 정확도로 꽤 훌륭한 성능을 내고 있다.
하지만 두 성능이 매우 비슷하므로 과소적합인 것 같다.
모델의 제약을 더 풀어주기 위해 C 를 증가시켜본다. 

In [ ]:
logreg100 = LogisticRegression(C=100, max_iter=5000).fit(X_train, y_train)
print("훈련 세트 점수: {:.3f}".format(logreg100.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg100.score(X_test, y_test)))

# 훈련, 테스트 둘 다 증가 
# 이는 복잡도가 높은 모델일수록 성능이 좋음을 말해준다. 

In [ ]:
logreg001 = LogisticRegression(C=0.01, max_iter=5000).fit(X_train, y_train)
print("훈련 세트 점수: {:.3f}".format(logreg001.score(X_train, y_train)))
print("테스트 세트 점수: {:.3f}".format(logreg001.score(X_test, y_test)))

# 규제를 더 강하게 하기 위해 C=0.01 사용 
# 과소적합된 모델, 정확도 낮아짐 

In [ ]:
# C 설정에 따른 모델의 계수 확인 
plt.plot(logreg100.coef_.T, '^', label="C=100")
plt.plot(logreg.coef_.T, 'o', label="C=1")
plt.plot(logreg001.coef_.T, 'v', label="C=0.001")
plt.xticks(range(cancer.data.shape[1]), cancer.feature_names, rotation=90)
xlims = plt.xlim()
plt.hlines(0, xlims[0], xlims[1])
plt.xlim(xlims)
plt.ylim(-5, 5)
plt.xlabel("특성")
plt.ylabel("계수 크기")
plt.legend()
plt.show() # 책에는 없음

> 그래프 해석 
+ 규제를 강하게 할수록 계수들을 0에 더 가깝게 만들지만 완전히 0이 되지는 않는다.
+ 세 번째 계수(mean perimeter)를 보면 C=100, C=1일때 이 계수는 음수지만, C=0.001일때 양수가 되며 C=1일때보다 절대값이 크다.
+ texture error 특성은 악성인 샘플과 관련이 깊다, 하지만 mean periment는 계수가 바뀌는 것을 보아 양성이나 악성의 신호 모두가 될 수 있다. 

더 이해하기 쉬운 모델을 원한다면 
L1 규제를 사용하는 것이 좋다.

### L1 규제 사용 

In [ ]:
for C, marker in zip([0.001, 1, 100], ['o', '^', 'v']):
    lr_l1 = LogisticRegression(solver='liblinear', C=C, penalty="l1", max_iter=1000).fit(X_train, y_train)
    # penalty : 어떤 규제를 사용할 지 결정 
    # L1 : 모든 규제 사용 x , 계수가 0이 될 수 있음/. 
    print("C={:.3f} 인 l1 로지스틱 회귀의 훈련 정확도: {:.2f}".format(
          C, lr_l1.score(X_train, y_train)))
    print("C={:.3f} 인 l1 로지스틱 회귀의 테스트 정확도: {:.2f}".format(
          C, lr_l1.score(X_test, y_test)))
    plt.plot(lr_l1.coef_.T, marker, label="C={:.3f}".format(C))

plt.xticks(range(cancer.data.shape[1]), cancer.feature_names, rotation=90)
xlims = plt.xlim()
plt.hlines(0, xlims[0], xlims[1])
plt.xlim(xlims)
plt.xlabel("특성")
plt.ylabel("계수 크기")

plt.ylim(-5, 5)
plt.legend(loc=3)
plt.show() # 책에는 없음